In [2]:
import numpy as np
from datetime import date, timedelta, datetime
import pandas as pd
import numpy as np
import requests
import math
import csv, re
import os
import matplotlib.pyplot as plt
import logging


logging.basicConfig(level='CRITICAL')
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.CRITICAL)
mpl_logger.disabled = True

result = pd.DataFrame()
stage = 0
response = []

# set stock ticker symbol
symbol = 'SBIN'
series = 'EQ'
 
# set date range for historical prices
end_date = date.today() # 24-03-2021
start_date = end_date - timedelta(days=364) # 24-03-2020
 
# reformat date range
end = end_date.strftime('%d-%m-%Y')
start = start_date.strftime('%d-%m-%Y')

newheaders = {
            'Connection': 'keep-alive',
            'Cache-Control': 'max-age=0',
            'DNT': '1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36',
            'Sec-Fetch-User': '?1',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-Mode': 'navigate',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
          }

url = "https://www.nseindia.com"
extendedLink = "/api/historical/cm/equity" + "?symbol=" + symbol + "&series=[%22" + series + "%22]&from=" + start + "&to=" + end +"&csv=true"
stkurl = url +  extendedLink 
# print(stkurl)

https://www.nseindia.com/api/historical/cm/equity?symbol=SBIN&series=[%22EQ%22]&from=29-03-2020&to=28-03-2021&csv=true


In [6]:

try:
    output = requests.get(url, headers=newheaders)
    s = requests.Session()
    output = s.get(url,headers=newheaders)
    output = s.get(stkurl, headers=newheaders)
except ValueError:
    print(output)
# print(output)

<Response [200]>


In [ ]:
data = output.content.decode('utf-8').splitlines()
# Open the file for writing
with open("stockdata.csv", "w", encoding='utf-8') as csv_file:
    # Create the writer object with tab delimiter
    writer = csv.writer(csv_file, delimiter = ',', quoting=csv.QUOTE_NONE, quotechar='',escapechar='\\')
    for line in data:
        writer.writerow(re.split('\s+',line))

In [47]:
df = pd.read_csv("stockdata.csv", delimiter=',', quotechar='"', escapechar='\\',usecols=['Date,\\','close,\\'])
df['Date'] = df['Date,\\']
df['Date'] = df.Date.str.replace("\\","")
df['Close'] = df['close,\\']
df['Close'] = df.Close.str.replace("\\","").astype(float)
prices = df.drop(['Date,\\', 'close,\\'], axis=1)
prices['returns'] = (np.log(prices.Close / prices.Close.shift(-1)))
count = int()
prices['vix'] = ""
# print(count)
samplerange = 10
vixlist = []
rank = []
# print(prices.loc[1:10])
while count < len(prices.Close):
    if count < len(prices.Close)-samplerange:
        p = prices.loc[count:(count+samplerange),]
        daily_std = round(np.std(p.returns, ddof=1), 6)
        std = daily_std * round(math.sqrt( samplerange ), 4)
        vix = np.round(std*100, 2)
        prices.loc[count:count,'vix'] = vix
    else:
        prices.loc[count:count,'vix'] = 0
    count += 1
# while count > 0:
#     if count > samplerange-1:
#         p = prices.loc[(count-samplerange):count,]
#         daily_std = round(np.std(p.returns, ddof=1), 6)
#         std = daily_std * round(math.sqrt( samplerange ), 4)
#         vix = np.round(std*100, 2)
#         prices.loc[count:count,'vix'] = vix
# #         vixlist = vix
# #         print(count)
#     else:
#         prices.loc[count:count,'vix'] = 0
# #         vixlist = 0
#     count -= 1

# print(rank)
#       #  Using at  is much faster than .loc howevr getting warning while using both in value assgnment
#         prices.loc[count,'vix'] = np.round(std*100, 2)
#         prices['vix'].values[count] = np.round(std*100, 2)
#         vixlist.append(np.round(std*100, 2))

# #tried one more method to add list to existing dataframe    
# prices['vix'] = vixlist
prices['vix_rank'] = round(prices['vix'].rank(pct=True)*100, 2)
# prices = prices.assign(vix = vixlist)
print(prices)

            Date   Close   returns   vix  vix_rank
0    26-Mar-2021  357.20  0.005615  5.13     12.45
1    25-Mar-2021  355.20 -0.013006  5.02     11.04
2    24-Mar-2021  359.85 -0.035087     5     10.44
3    23-Mar-2021  372.70  0.015412  4.47      7.23
4    22-Mar-2021  367.00 -0.011244  4.77      9.24
..           ...     ...       ...   ...       ...
244  07-Apr-2020  186.40  0.060256     0      2.21
245  03-Apr-2020  175.50 -0.061060     0      2.21
246  01-Apr-2020  186.55 -0.053743     0      2.21
247  31-Mar-2020  196.85  0.051868     0      2.21
248  30-Mar-2020  186.90       NaN     0      2.21

[249 rows x 5 columns]


In [ ]:
# print(prices.loc['07-04-2020':'01-04-2020'])
# print(prices['Date'])
# print(prices)
count = int()
# print(len(prices))
# while count < len(prices):
#     print(prices[count])
#     count += 1
# for i in enumerate(prices[10:]
# for startdt in enumerate(prices[10:]):
#     print('start: ', startdt)

# for startdt, enddt in zip(prices['Date'], prices[10:]['Date']):
#     count += 1
#     if count > 10:
#         try:
#             print('start: ', startdt,'end: ', enddt)
#             print(prices.loc[startdt:enddt]['Close'])
#         except AttributeError:
#             print(enddt)
        

# print(prices)

# mask = (prices['date'] > '2020-6-1') & (prices['date'] <= '2020-6-10')
# print(prices.loc[mask])


# print(prices[prices.date.between(start_date, end_date)])

# print(prices)

# for date in prices['Date']:
# mask = ((prices['Date']=='23-03-2021'))
# print(prices.loc[prices['Date']<='01-03-2021'])
# oidata.loc[oidata['strikePrice'] == atm_ce_ii]['impliedVolatility_ce'].values[0]


In [ ]:
prices['Date'] = pd.to_datetime( prices['Date'] )
# prices['Close'] = prices['Close'].astype(float)
prices.sort_values(by=['Date'], inplace=True, ascending=True)
# print(prices)

# calculate daily logarithmic return
prices['returns'] = (np.log(prices.Close / prices.Close.shift(-1)))

# calculate daily standard deviation of returns
# NumPy's std calculates the population standard deviation by default (it is behaving like Excel's STDEVP).
# To make NumPy's std function behave like Excel's STDEV, pass in the value ddof=1:
daily_std = round(np.std(prices.returns, ddof=1), 6)

# annualized daily standard deviation
samples = len(prices.index)-1
std = daily_std * round(math.sqrt( samples ), 4)
vix = np.round(std*100, 2)
print(vix)

In [ ]:
# Plot histograms
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
n, bins, patches = ax.hist(prices.returns.values, bins=50, alpha=0.65, color='blue')
 
ax.set_xlabel('log return of stock price')
ax.set_ylabel('frequency of log return')
ax.set_title('Historical Volatility for ' + symbol)
  
# get x and y coordinate limits
x_corr = ax.get_xlim()
y_corr = ax.get_ylim()
  
# make room for text
header = y_corr[1] / 5
y_corr = (y_corr[0], y_corr[1] + header)
ax.set_ylim(y_corr[0], y_corr[1])
 
# print historical volatility on plot
x = x_corr[0] + (x_corr[1] - x_corr[0]) / 30
y = y_corr[1] - (y_corr[1] - y_corr[0]) / 15
ax.text(x, y , 'Annualized Volatility: ' + str(vix)+'%', fontsize=11, fontweight='bold')
x = x_corr[0] + (x_corr[1] - x_corr[0]) / 15
y -= (y_corr[1] - y_corr[0]) / 20
 
# save histogram plot of historical price volatility
fig.tight_layout()
fig.savefig('historical volatility.png')
